In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("ner.csv", encoding = "ISO-8859-1", on_bad_lines= "skip")
data

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050790,1048570,they,respond,to,TO,lowercase,to,VBD,lowercase,responded,...,forc,NNS,lowercase,forces,lowercase,said,47959.0,lowercase,they,O
1050791,1048571,respond,to,the,DT,lowercase,the,TO,lowercase,to,...,said,VBD,lowercase,said,lowercase,they,47959.0,lowercase,responded,O
1050792,1048572,to,the,attack,NN,lowercase,attack,DT,lowercase,the,...,they,PRP,lowercase,they,lowercase,responded,47959.0,lowercase,to,O
1050793,1048573,the,attack,with,IN,lowercase,with,NN,lowercase,attack,...,respond,VBD,lowercase,responded,lowercase,to,47959.0,lowercase,the,O


In [2]:
STATE_PROB = {}
SYMBOL_PROB = {}

In [3]:
for _ in range(len(data)-1):
    if data.loc[_, "sentence_idx"] == data.loc[_+1, "sentence_idx"]:
        curr_pos = data.loc[_, "next-next-pos"]
        next_pos = data.loc[_+1, 'next-next-pos']
        if curr_pos not in STATE_PROB:
            STATE_PROB[curr_pos] = {}
        if next_pos not in STATE_PROB[curr_pos]:
            STATE_PROB[curr_pos][next_pos] = 0
        STATE_PROB[curr_pos][next_pos] += 1
        
        curr_word = data.loc[_, "next-next-lemma"]
        if curr_pos not in SYMBOL_PROB:
            SYMBOL_PROB[curr_pos] = {}
        if curr_word not in SYMBOL_PROB[curr_pos]:
            SYMBOL_PROB[curr_pos][curr_word] = 0
        SYMBOL_PROB[curr_pos][curr_word] += 1

In [4]:
for _ in STATE_PROB:
    count = 0
    for __ in STATE_PROB[_]:
        count += STATE_PROB[_][__]
    
    for __ in STATE_PROB[_]:
        STATE_PROB[_][__] /= count
        
for _ in SYMBOL_PROB:
    count = 0
    for __ in SYMBOL_PROB[_]:
        count += SYMBOL_PROB[_][__]
    
    for __ in SYMBOL_PROB[_]:
        SYMBOL_PROB[_][__] /= count

        

In [11]:
REVERSE_SYMBOL_PROB = {}

for _ in SYMBOL_PROB:
    for __ in SYMBOL_PROB[_]:
        if __ not in REVERSE_SYMBOL_PROB:
            REVERSE_SYMBOL_PROB[__] = {}
        REVERSE_SYMBOL_PROB[__][_] = SYMBOL_PROB[_][__]

In [43]:
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

Lemmatizer = WordNetLemmatizer()

def pos_tag(sentence):

    words = word_tokenize(sentence.lower())
    words = [Lemmatizer.lemmatize(word) for word in words]

    temp = [(REVERSE_SYMBOL_PROB[words[0]][_], _) for _ in REVERSE_SYMBOL_PROB[words[0]]]
    PROB = sorted(temp, reverse = True)[0][0]
    TAGS = [sorted(temp, reverse = True)[0][1]]
    
    for _ in range(1, len(words)):
        temp = []
        for next_tag in STATE_PROB[TAGS[-1]]:
            try:
                temp.append((PROB * STATE_PROB[TAGS[-1]][next_tag] * SYMBOL_PROB[next_tag][words[_]], next_tag))
            except:
                temp.append((0, next_tag))
        PROB, next_tag = sorted(temp, reverse = True)[0]
        TAGS.append(next_tag)

    return TAGS, PROB
        

In [53]:
x = "I want to book a flight from Delhi to Mumbai"

In [54]:
pos_tag(x)

(['PRP', 'VBZ', 'TO', 'NN', 'DT', 'NN', 'IN', 'NNP', 'TO', 'NNP'],
 6.8274555975393086e-31)